<a href="https://colab.research.google.com/github/lucasfreire01/Cancer_pele/blob/main/Cancerpele.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
#Starting import libraries
import warnings
warnings.filterwarnings('ignore')

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [39]:
#Loading the train dataset 
train = pd.read_csv('/content/drive/MyDrive/Cancer_pele/train_metadata_pPa7YD3.csv')
train

,filename,age,sex,body_site,melanoma_history,breslow,ulceration,resolution,tif_cksum,tif_size,us_tif_url,eu_tif_url,as_tif_url
0,1u4lhlqb.tif,[32:34[,2,thigh,YES,<0.8,NO,0.264384,3028450373,747151312,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...
1,rqumqnfp.tif,[46:48[,1,trunc,NO,[1 : 2[,NO,0.264384,1294832049,591027450,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...
2,bu5xt1xm.tif,[64:66[,2,face,NO,<0.8,NO,0.264384,774102360,465947458,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...
3,dibvu7wk.tif,[62:64[,2,forearm,NaN,[2 : 4[,YES,0.226490,515827065,568174704,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...
4,qsza4coh.tif,[90:92[,2,face,NO,[1 : 2[,NO,0.226490,1541795099,1042691978,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337,n7jd638y.tif,[68:70[,1,head/neck,NaN,[1 : 2[,NO,0.250000,1949851255,136729436,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...
1338,8wg601oe.tif,[66:68[,1,head/neck,NaN,<0.8,NaN,0.250000,1344207528,1603484314,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...
1339,1ejfd01b.tif,[80:82[,1,trunk,NaN,>=4,YES,0.250000,4292211076,1423564688,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...
1340,2ztrjp20.tif,[42:44[,2,trunk,NaN,<0.8,NaN,0.250000,2949914817,336489430,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...


In [40]:
#Loadinig the train_labels to take the relapse column to use as target
train_labels = pd.read_csv('/content/drive/MyDrive/Cancer_pele/train_labels.csv')
train_labels

,filename,relapse
0,1u4lhlqb.tif,0
1,rqumqnfp.tif,0
2,bu5xt1xm.tif,0
3,dibvu7wk.tif,0
4,qsza4coh.tif,0
...,...,...
1337,n7jd638y.tif,0
1338,8wg601oe.tif,0
1339,1ejfd01b.tif,0
1340,2ztrjp20.tif,0


In [41]:
#See the distribuation of values so if relapse is 0 there is relapse if the values is 1 no had relapse
train_labels.relapse.value_counts()

0    1129
1     213
Name: relapse, dtype: int64

In [42]:
#See the distruation at melanoma_historiry 
train['melanoma_history'].value_counts()

NO     585
YES     57
Name: melanoma_history, dtype: int64

In [43]:
#take the missing values and put this place the value YES for have a distribuation more balance
train['melanoma_history'].fillna('YES', inplace=True)

In [44]:
#This is the result of distribuation
train['melanoma_history'].value_counts()

YES    757
NO     585
Name: melanoma_history, dtype: int64

In [45]:
#Take the first age in each line and transfom in int
train['age_int'] = train['age'].str.extract(r'(\d+)').astype(int)

In [46]:
train

,filename,age,sex,body_site,melanoma_history,breslow,ulceration,resolution,tif_cksum,tif_size,us_tif_url,eu_tif_url,as_tif_url,age_int
0,1u4lhlqb.tif,[32:34[,2,thigh,YES,<0.8,NO,0.264384,3028450373,747151312,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...,32
1,rqumqnfp.tif,[46:48[,1,trunc,NO,[1 : 2[,NO,0.264384,1294832049,591027450,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...,46
2,bu5xt1xm.tif,[64:66[,2,face,NO,<0.8,NO,0.264384,774102360,465947458,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...,64
3,dibvu7wk.tif,[62:64[,2,forearm,YES,[2 : 4[,YES,0.226490,515827065,568174704,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...,62
4,qsza4coh.tif,[90:92[,2,face,NO,[1 : 2[,NO,0.226490,1541795099,1042691978,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337,n7jd638y.tif,[68:70[,1,head/neck,YES,[1 : 2[,NO,0.250000,1949851255,136729436,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...,68
1338,8wg601oe.tif,[66:68[,1,head/neck,YES,<0.8,NaN,0.250000,1344207528,1603484314,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...,66
1339,1ejfd01b.tif,[80:82[,1,trunk,YES,>=4,YES,0.250000,4292211076,1423564688,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...,80
1340,2ztrjp20.tif,[42:44[,2,trunk,YES,<0.8,NaN,0.250000,2949914817,336489430,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...,42


In [47]:
#Using the One_hot_encoder to transform the melanoma_history column in two binary column
from sklearn.preprocessing import OneHotEncoder
one = OneHotEncoder()
one_hot = pd.get_dummies(train['melanoma_history'])
one_hot

,NO,YES
0,0,1
1,1,0
2,1,0
3,0,1
4,1,0
...,...,...
1337,0,1
1338,0,1
1339,0,1
1340,0,1


In [48]:
#take the most important column this column is the same the test column
train_new = train[['filename', 'age', 'sex', 'body_site', 'melanoma_history',
       'resolution']]

In [49]:
train_new

,filename,age,sex,body_site,melanoma_history,resolution
0,1u4lhlqb.tif,[32:34[,2,thigh,YES,0.264384
1,rqumqnfp.tif,[46:48[,1,trunc,NO,0.264384
2,bu5xt1xm.tif,[64:66[,2,face,NO,0.264384
3,dibvu7wk.tif,[62:64[,2,forearm,YES,0.226490
4,qsza4coh.tif,[90:92[,2,face,NO,0.226490
...,...,...,...,...,...,...
1337,n7jd638y.tif,[68:70[,1,head/neck,YES,0.250000
1338,8wg601oe.tif,[66:68[,1,head/neck,YES,0.250000
1339,1ejfd01b.tif,[80:82[,1,trunk,YES,0.250000
1340,2ztrjp20.tif,[42:44[,2,trunk,YES,0.250000


In [50]:
train_new.drop('filename', axis=1, inplace=True)

In [51]:
#out the age with the first value as int and the melanoma with one_hot_encoder 
train_new['age'] = train['age_int']
train_new.drop('melanoma_history', axis=1, inplace=True)
train_new['melanoma_NO'] = one_hot['NO']
train_new['melanoma_YES'] = one_hot['YES']
train_new

,age,sex,body_site,resolution,melanoma_NO,melanoma_YES
0,32,2,thigh,0.264384,0,1
1,46,1,trunc,0.264384,1,0
2,64,2,face,0.264384,1,0
3,62,2,forearm,0.226490,0,1
4,90,2,face,0.226490,1,0
...,...,...,...,...,...,...
1337,68,1,head/neck,0.250000,0,1
1338,66,1,head/neck,0.250000,0,1
1339,80,1,trunk,0.250000,0,1
1340,42,2,trunk,0.250000,0,1


In [52]:
train_new.isnull().sum()

age              0
sex              0
body_site       18
resolution       0
melanoma_NO      0
melanoma_YES     0
dtype: int64

In [53]:
#treating missing value at body_site column that is categorical column
mode = train_new['body_site'].mode().values[0]
train_new['body_site'].fillna(mode, inplace=True)

In [54]:
#See the missing values
train_new.isnull().sum()

age             0
sex             0
body_site       0
resolution      0
melanoma_NO     0
melanoma_YES    0
dtype: int64

In [55]:
#let the column with tow decimal places
train_new['resolution'] = train_new['resolution'].round(2)

In [56]:
train_new

,age,sex,body_site,resolution,melanoma_NO,melanoma_YES
0,32,2,thigh,0.26,0,1
1,46,1,trunc,0.26,1,0
2,64,2,face,0.26,1,0
3,62,2,forearm,0.23,0,1
4,90,2,face,0.23,1,0
...,...,...,...,...,...,...
1337,68,1,head/neck,0.25,0,1
1338,66,1,head/neck,0.25,0,1
1339,80,1,trunk,0.25,0,1
1340,42,2,trunk,0.25,0,1


In [57]:
bd = train_new['body_site']
train_new['body_site'].value_counts()

trunc                  324
leg                    163
arm                    149
face                   134
trunk                  132
lower limb/hip         114
head/neck               71
upper limb/shoulder     70
thigh                   58
forearm                 26
foot                    23
hand/foot/nail          21
sole                    13
neck                    12
toe                      7
hand                     7
finger                   6
seat                     6
scalp                    4
nail                     2
Name: body_site, dtype: int64

In [58]:
#Take the target column and put in the test variable
test_new = train_labels['relapse']

In [59]:
#Using the One_hot_encoder in other categorical column
encoded_bd = pd.DataFrame(one.fit_transform(train_new[['body_site']]).toarray())
categories = train_new['body_site'].unique()
encoded_bd.columns = categories

In [60]:
train_new = pd.concat([train_new, encoded_bd], axis=1)

In [61]:
train_new = train_new.drop('body_site', axis=1)

In [62]:
#Normalization the dataset
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_new = pd.DataFrame(scaler.fit_transform(train_new), columns=train_new.columns)

In [63]:
test_new = test_new.astype('float32')
test_new1 = tf.convert_to_tensor(test_new)
train_new1 = tf.convert_to_tensor(train_new)

In [85]:
#Spliting the two dataset train and test in x_train,x_test, y_train, y_train
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_new,test_new, test_size=0.2, random_state=42, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [86]:
#Build the model very simple with the keras.Sequential and the dense for switch between leyers, just with a entry leyers, one hiddren leyer and 
#one output leyer. The hiddren leyer have 64 neurons and just one neurons output the activation at hiddren leyer is the popular relu the target have 
# values between 0 and 1 so the sigmiod was used to be the activation at out oyt leyer and then will have a treate to transform in 0 and 1
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(64, input_dim=25, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


#Compiling the model with loss= 'binary_crossentropy, optimizer='adam', metrics=['accuracy']
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#training the model and validation it
model.fit(X_train, y_train, epochs=150, batch_size=33, validation_data=(X_test, y_test))

#take the precision of model and next treating the predict to have these values in 0 and 1
loss, accuracy = model.evaluate(X_test, y_test)
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

Epoch 1/150
26/26 [==============================] - 1s 10ms/step - loss: 0.5936 - accuracy: 0.7914 - val_loss: 0.5169 - val_accuracy: 0.8587
Epoch 2/150
26/26 [==============================] - 0s 4ms/step - loss: 0.4845 - accuracy: 0.8415 - val_loss: 0.4403 - val_accuracy: 0.8587
Epoch 3/150
26/26 [==============================] - 0s 3ms/step - loss: 0.4410 - accuracy: 0.8415 - val_loss: 0.4140 - val_accuracy: 0.8587
Epoch 4/150
26/26 [==============================] - 0s 3ms/step - loss: 0.4271 - accuracy: 0.8415 - val_loss: 0.4075 - val_accuracy: 0.8587
Epoch 5/150
26/26 [==============================] - 0s 4ms/step - loss: 0.4206 - accuracy: 0.8415 - val_loss: 0.4046 - val_accuracy: 0.8587
Epoch 6/150
26/26 [==============================] - 0s 3ms/step - loss: 0.4164 - accuracy: 0.8415 - val_loss: 0.4028 - val_accuracy: 0.8587
Epoch 7/150
26/26 [==============================] - 0s 3ms/step - loss: 0.4125 - accuracy: 0.8415 - val_loss: 0.4022 - val_accuracy: 0.8587
Epoch 8/150


In [67]:
y_pred = y_pred.reshape(1,-1)

In [68]:
y_pred

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0]])

In [69]:
#See the mean absolute erro
mae = model.evaluate(X_test, y_test)

print("MAE on test set:", mae)

9/9 [==============================] - 0s 3ms/step - loss: 0.4046 - accuracy: 0.8625
MAE on test set: [0.4046211540699005, 0.8624535202980042]


In [70]:
#dataset to validation
val = pd.read_csv('/content/test_metadata.csv')
val

,filename,age,sex,body_site,melanoma_history,resolution
0,qpbyhjj8.tif,[34:36[,2,trunc,NO,0.121399
1,8tn0wx0q.tif,[78:80[,1,arm,NO,0.250000


In [71]:
val = val.drop('filename', axis=1)

In [72]:
#preprocess this validation values
one_hot = pd.get_dummies(train['melanoma_history'])
one_hot

val['age_int'] = val['age'].str.extract(r'(\d+)').astype(int)
val['age'] = train['age_int']
val.drop('melanoma_history', axis=1, inplace=True)
val['melanoma_NO'] = one_hot['NO']
val['melanoma_YES'] = one_hot['YES']
val

,age,sex,body_site,resolution,age_int,melanoma_NO,melanoma_YES
0,32,2,trunc,0.121399,34,0,1
1,46,1,arm,0.250000,78,1,0


In [73]:
encoded_bd = pd.DataFrame(one.fit_transform(val[['body_site']]).toarray())
categories = val['body_site'].unique()
encoded_bd.columns = categories

In [74]:
val['trunc'] = encoded_bd['trunc']
val['arm'] = encoded_bd['arm']
val = val.drop('body_site', axis=1)
val = val.drop('age', axis=1)

In [75]:
val = pd.DataFrame(scaler.fit_transform(val), columns=val.columns)
val

,sex,resolution,age_int,melanoma_NO,melanoma_YES,trunc,arm
0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
1,0.0,1.0,1.0,1.0,0.0,1.0,0.0


In [76]:
bd_list = []
for i in bd.value_counts():
  bd_list.append(i)

In [77]:
#take the  body site column at train  dataset for put the names in validation dataset but this column add no have values
bd1 = bd.to_list()
bd1= list(set(bd1))

In [78]:
val.columns

Index(['sex', 'resolution', 'age_int', 'melanoma_NO', 'melanoma_YES', 'trunc',
       'arm'],
      dtype='object')

In [79]:
for i in bd1:
  if i not in val.columns:
    val[i] = 0

In [80]:
val.columns

Index(['sex', 'resolution', 'age_int', 'melanoma_NO', 'melanoma_YES', 'trunc',
       'arm', 'foot', 'scalp', 'seat', 'forearm', 'lower limb/hip', 'hand',
       'trunk', 'neck', 'thigh', 'head/neck', 'leg', 'toe', 'face',
       'upper limb/shoulder', 'finger', 'sole', 'hand/foot/nail', 'nail'],
      dtype='object')

In [81]:
y_pred_prob = model.predict(val)
y_pred = (y_pred_prob > 0.5).astype(int)

1/1 [==============================] - 0s 33ms/step


In [82]:
y_pred

array([[0],
       [0]])

In [83]:
val['relapse'] = y_pred

In [84]:
#This is the result
val

,sex,resolution,age_int,melanoma_NO,melanoma_YES,trunc,arm,foot,scalp,seat,...,head/neck,leg,toe,face,upper limb/shoulder,finger,sole,hand/foot/nail,nail,relapse
0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
#Thank you to see my code have a good day